# "GitHub repositories BigGraph and users recommendations by embeddings" project

In [1]:
from resources.utils import *

DUMP_DATE = '2019-06'

## Download data

Run `db_download.sh` script to download dump

extract needed tables

In [2]:
!tar -C data/ -xvzf data/mysql-{DUMP_DATE}-01.tar.gz mysql-{DUMP_DATE}-01/project_members.csv mysql-{DUMP_DATE}-01/followers.csv mysql-{DUMP_DATE}-01/watchers.csv mysql-{DUMP_DATE}-01/projects.csv mysql-{DUMP_DATE}-01/users.csv

mysql-2019-06-01/followers.csv
mysql-2019-06-01/watchers.csv
mysql-2019-06-01/projects.csv
mysql-2019-06-01/project_members.csv
mysql-2019-06-01/users.csv


## Dataset preprocessing for project

The goal of this stage is to process the GitHub dataset from http://ghtorrent.org/ project into directed graph. <br>
The source data consists from .csv files which represent tabular SQL data. Each file is one table in source database. <br>
Total size of the dataset is 100GB. The size of the tables used for graph building is ~7GB.
*Database schema:*
![image.png](http://ghtorrent.org/files/schema.png)

The following tables are used for the graph representation:
* Followers
* Watchers
* Project Members

##### Graph structure 
Following nodes are defined as points of interest for this project:
* Users
* Projects

Types of graph edges:
* Follows
* Watches
* Is member of

Possible edges:
* User => follow => User
* User => follow => Project
* User => watch => User
* User => watch => Project
* User => is member => Project

In [2]:
from pyspark import SparkContext, SparkConf, SQLContext

# constants
DATA_FOLDER = f"./data/mysql-{DUMP_DATE}-01/"
FOLLOWERS_PATH = DATA_FOLDER + "followers.csv"
WATCHERS_PATH = DATA_FOLDER + "watchers.csv"
PROJECT_MEMBERS_PATH = DATA_FOLDER + "project_members.csv"

# relations constants
IS_MEMBER_OF = "is_member_of"
FOLLOWS = "follows"
WATCHES = "watches"

In [2]:
conf = SparkConf()
sc = SparkContext(conf=conf)

### Process SQL structured data into edges

In [3]:
members_rdd = sc.textFile(PROJECT_MEMBERS_PATH).map(lambda x: x.split(",")[:-2])

def addIdentifiers(row, id_1, id_2):
    '''
    Adds identifier for each id to distinguish them
    '''
    row[0] = id_1 + row[0]
    row[1] = id_2 + row[1]
    return row

def swapColumns(row):
    temp = row[0]
    row[0] = row[1]
    row[1] = temp
    return row

def addRelation(row, relation):
    temp = row[1]
    row[1] = relation
    row.append(temp)
    return row

# Processing of members table
members_rdd = members_rdd.map(lambda x: addIdentifiers(x, "repo_id_", "user_id_")).map(swapColumns).map(lambda x: addRelation(x,IS_MEMBER_OF))
members_rdd.take(5)

[['user_id_1', 'is_member_of', 'repo_id_1'],
 ['user_id_2', 'is_member_of', 'repo_id_1'],
 ['user_id_4', 'is_member_of', 'repo_id_1'],
 ['user_id_24', 'is_member_of', 'repo_id_3'],
 ['user_id_5465', 'is_member_of', 'repo_id_3']]

In [4]:
# Processing of followers table
followers_rdd = sc.textFile(FOLLOWERS_PATH).map(lambda x: x.split(",")[:-1]) \
    .map(lambda x: addIdentifiers(x, "user_id_", "user_id_")) \
    .map(swapColumns) \
    .map(lambda x: addRelation(x, FOLLOWS))

followers_rdd.take(5)

[['user_id_2', 'follows', 'user_id_1'],
 ['user_id_4', 'follows', 'user_id_1'],
 ['user_id_17896', 'follows', 'user_id_1'],
 ['user_id_21523', 'follows', 'user_id_1'],
 ['user_id_29121', 'follows', 'user_id_1']]

In [5]:
# Processing of watchers table
watchers_rdd = sc.textFile(WATCHERS_PATH).map(lambda x: x.split(",")[:-1]) \
    .map(lambda x: addIdentifiers(x, "repo_id_", "user_id_")) \
    .map(swapColumns) \
    .map(lambda x: addRelation(x, WATCHES))

watchers_rdd.take(5)

[['user_id_1', 'watches', 'repo_id_1'],
 ['user_id_2', 'watches', 'repo_id_1'],
 ['user_id_4', 'watches', 'repo_id_1'],
 ['user_id_6', 'watches', 'repo_id_1'],
 ['user_id_7', 'watches', 'repo_id_1']]

### Split into train, validation and test datasets

In [ ]:
# merge into one entity
rdd = members_rdd.union(followers_rdd).union(watchers_rdd)

sqlContext = SQLContext(sc)
df = sqlContext.createDataFrame(rdd, ['from_entity_id', 'relation', 'to_entity_id'])
df.coalesce(1).write.option("delimiter", "\t").format('com.databricks.spark.csv').options(header='true').save(DATA_FOLDER + 'graph_edges')
shuffledDF = df.randomSplit([0.1, 0.1, 0.8])
val_set = shuffledDF[0]
test_set = shuffledDF[1]
train_set = shuffledDF[2]

# save into files
val_set.coalesce(1).write.option("delimiter", "\t").format('com.databricks.spark.csv').options(header='true').save(DATA_FOLDER + 'graph_edges/validation')
test_set.coalesce(1).write.option("delimiter", "\t").format('com.databricks.spark.csv').options(header='true').save(DATA_FOLDER + 'graph_edges/test')
train_set.coalesce(1).write.option("delimiter", "\t").format('com.databricks.spark.csv').options(header='true').save(DATA_FOLDER + 'graph_edges/train')

## Store metadata

Metainfo we store in MongoDB

In [ ]:
def convert_dt(sdt):
    try:
        return pd.datetime.strptime(sdt, '%Y-%m-%d %H:%M:%S')
    except Exception as e:
        print(e)

### users

In [5]:
users_df = pd.read_csv(f'{DATA_FOLDER}users.csv',
                       header=None,
                       names=['id', 'login', 'company', 'created_at', 'type', 'fake', 'deleted', 'long', 'lat',
                              'country_code', 'state', 'city', 'location'],
#                        parse_dates=[3],
#                        quoting=1,
#                        quotechar='"',
#                        encoding='utf-8',
                       low_memory=False,
                       error_bad_lines=False).replace({'\\N': None}).set_index('id')
print(users_df.shape)
users_df['created_at'] = users_df['created_at'].map(convert_dt)
users_df = users_df[~users_df['created_at'].isna()]
users_df['fake'] = users_df['fake'].replace({1: True, 0: False, '1': True, '0': False}).astype(bool)
users_df['deleted'] = users_df['deleted'].replace({1: True, 0: False, '1': True, '0': False}).astype(bool)
users_df['long'] = users_df['long'].astype(float)
users_df['lat'] = users_df['lat'].astype(float)
users_df.shape

(30600249, 12)

In [3]:
mc['github']['users'].create_index('login')

'login_1'

In [6]:
to_insert = []

for uid, row in tn(users_df.iterrows(), total=users_df.shape[0]):
    to_insert.append({'_id': uid, **row.to_dict()})
    if len(to_insert) >= 10_000:
        mc['github']['users'].insert_many(to_insert)
        to_insert = []
if len(to_insert):
    mc['github']['users'].insert_many(to_insert)
mc['github']['users'].estimated_document_count()

30600249

### projects (repos)

In [ ]:
for field in ['name', 'owner_id', 'url']:
    mc['github']['projects'].create_index(field)

In [7]:
try:
    stop = False
    skip_count = 0
    for i in range(1000):
        print(i)
        repos_df = pd.read_csv(f'{DATA_FOLDER}projects.csv',
                               skiprows=10_000_000*i,
                               nrows=10_000_000,
                               header=None,
                               na_values=['\\N'],
                               names=["id", "url", "owner_id", "name", "description", "language",
                                      "created_at", "forked_from", "deleted", "updated_at", "?"],
                               error_bad_lines=False,
                               low_memory=False,
                              ).set_index('id')
        c = repos_df.shape[0]
        if c < 10_000_000:
            stop = True
        repos_df['created_at'] = repos_df['created_at'].map(convert_dt)
        repos_df = repos_df[~repos_df['created_at'].isna()]
        repos_df['updated_at'] = repos_df['updated_at'].map(convert_dt)
        repos_df = repos_df[~repos_df['updated_at'].isna()]
        skip_count += (c - repos_df.shape[0])
        repos_df.index = repos_df.index.astype(int)
        repos_df['url'] = repos_df['url'].astype(str).map(lambda x: x[29:])
        repos_df['deleted'] = repos_df['deleted'].replace({1: True, 0: False, 1: True, 0: False}).astype(bool)
        repos_df['owner_id'] = repos_df['owner_id'].astype(int)
        repos_df['forked_from'] = repos_df['forked_from'].astype(float)
        repos_df['?'] = repos_df['?'].astype(float)

        to_insert = []

        for rid, row in tn(repos_df.iterrows(), total=repos_df.shape[0]):
            to_insert.append({'_id': rid, **row.to_dict()})
            if len(to_insert) >= 10_000:
                mc['github']['projects'].insert_many(to_insert)
                to_insert = []
        if len(to_insert):
            mc['github']['project'].insert_many(to_insert)
        if stop:
            break
    m = f"[projects]: stored {mc['github']['projects'].estimated_document_count()} projects, {skip_count} skiped"
    tgn(m)
except Exception as e:
    m = f'[projects]: error - {e}'
    tgn(m)
m

'[projects]: stored 116010000 projects, 135185 skiped'

## Training

See `graph_config.py` config file for details (`torchbiggraph_train -h` for help)

In [ ]:
!torchbiggraph_train graph_config.py

Evaluating on validation

In [ ]:
!torchbiggraph_eval graph_config_validate.py

## Prepare tensorboard

In [2]:
# !torchbiggraph_export_to_tsv --dict {DATA_FOLDER}/graph_edges/dictionary.json \
# --checkpoint {DATA_FOLDER}/graph_edges/ --out {DATA_FOLDER}/graph_edges/embeddings.tsv

Loading relation types and entities...
Loading model check point...
Writing embeddings for entity type all partition 0...
- Processed 5000 entities so far...
- Processed 10000 entities so far...
- Processed 15000 entities so far...
- Processed 20000 entities so far...
- Processed 25000 entities so far...
- Processed 30000 entities so far...
- Processed 35000 entities so far...
- Processed 40000 entities so far...
- Processed 45000 entities so far...
- Processed 50000 entities so far...
- Processed 55000 entities so far...
- Processed 60000 entities so far...
- Processed 65000 entities so far...
- Processed 70000 entities so far...
- Processed 75000 entities so far...
- Processed 80000 entities so far...
- Processed 85000 entities so far...
- Processed 90000 entities so far...
- Processed 95000 entities so far...
- Processed 100000 entities so far...
- Processed 105000 entities so far...
- Processed 110000 entities so far...
- Processed 115000 entities so far...
- Processed 120000 entit

- Processed 1070000 entities so far...
- Processed 1075000 entities so far...
- Processed 1080000 entities so far...
- Processed 1085000 entities so far...
- Processed 1090000 entities so far...
- Processed 1095000 entities so far...
- Processed 1100000 entities so far...
- Processed 1105000 entities so far...
- Processed 1110000 entities so far...
- Processed 1115000 entities so far...
- Processed 1120000 entities so far...
- Processed 1125000 entities so far...
- Processed 1130000 entities so far...
- Processed 1135000 entities so far...
- Processed 1140000 entities so far...
- Processed 1145000 entities so far...
- Processed 1150000 entities so far...
- Processed 1155000 entities so far...
- Processed 1160000 entities so far...
- Processed 1165000 entities so far...
- Processed 1170000 entities so far...
- Processed 1175000 entities so far...
- Processed 1180000 entities so far...
- Processed 1185000 entities so far...
- Processed 1190000 entities so far...
- Processed 1195000 entit

- Processed 415000 entities so far...
- Processed 420000 entities so far...
- Processed 425000 entities so far...
- Processed 430000 entities so far...
- Processed 435000 entities so far...
- Processed 440000 entities so far...
- Processed 445000 entities so far...
- Processed 450000 entities so far...
- Processed 455000 entities so far...
- Processed 460000 entities so far...
- Processed 465000 entities so far...
- Processed 470000 entities so far...
- Processed 475000 entities so far...
- Processed 480000 entities so far...
- Processed 485000 entities so far...
- Processed 490000 entities so far...
- Processed 495000 entities so far...
- Processed 500000 entities so far...
- Processed 505000 entities so far...
- Processed 510000 entities so far...
- Processed 515000 entities so far...
- Processed 520000 entities so far...
- Processed 525000 entities so far...
- Processed 530000 entities so far...
- Processed 535000 entities so far...
- Processed 540000 entities so far...
- Processed 

- Processed 1485000 entities so far...
- Processed 1490000 entities so far...
- Processed 1495000 entities so far...
- Processed 1500000 entities so far...
- Processed 1505000 entities so far...
- Processed 1510000 entities so far...
- Processed 1515000 entities so far...
- Processed 1520000 entities so far...
- Processed 1525000 entities so far...
- Processed 1530000 entities so far...
- Processed 1535000 entities so far...
- Processed 1540000 entities so far...
- Processed 1545000 entities so far...
- Processed 1550000 entities so far...
- Processed 1555000 entities so far...
- Processed 1560000 entities so far...
- Processed 1565000 entities so far...
- Processed 1570000 entities so far...
- Processed 1575000 entities so far...
- Processed 1580000 entities so far...
- Processed 1585000 entities so far...
- Processed 1590000 entities so far...
- Processed 1595000 entities so far...
- Processed 1600000 entities so far...
- Processed 1605000 entities so far...
- Processed 1610000 entit

KeyboardInterrupt: 

slow ↑ method

In [21]:
!pip freeze | grep h5py

h5py==2.9.0


In [3]:
import json
import h5py
import os

!mkdir -p tb
!mkdir -p tb/embeddings

def dropCategories(df, col, threshold):
    drop_list = df[col].value_counts()[threshold:].index.to_list()
    df[col].cat.remove_categories(drop_list, inplace=True)
    return df

For faster results let's see only on repos with 1K+ stars

In [4]:
good_rids = pd.read_pickle('./tb/embeddings/repos_19k_gte1k_old.pkl').index.tolist()
len(good_rids)

18670

Read metadata

In [5]:
repos_cur = mc['github']['projects'].find({"_id": {"$in": good_rids}}, ['url', 'language', 'created_at', 'updated_at'])
repos_df = pd.DataFrame(list(repos_cur)).set_index('_id')[['url', 'language', 'created_at', 'updated_at']]
repos_df.language = repos_df.language.astype("category")
repos_df = dropCategories(repos_df, "language", 42)
repos_df

,url,language,created_at,updated_at
_id,,,,
6,cocos2d/cocos2d-x,C++,2010-11-18 23:17:00,2019-02-26 20:51:59
19,samuelclay/NewsBlur,Objective-C,2009-01-05 14:00:43,2019-02-26 03:37:41
29,opencv/opencv,C++,2012-07-19 09:40:17,2016-10-11 14:11:27
34,pockethub/PocketHub,JavaScript,2011-09-08 16:52:50,2016-10-11 14:11:27
37,angular/angular.js,JavaScript,2010-01-06 00:34:37,2019-02-27 03:14:26
49,alexgorbatchev/syntaxhighlighter,NaN,2014-10-07 13:48:58,2019-01-07 05:02:27
55,yui/yui3,JavaScript,2008-12-05 19:12:24,2019-02-26 04:45:11
71,llvm-mirror/llvm,NaN,2012-01-27 23:49:56,2019-02-26 12:59:44
74,joyent/libuv,C,2011-03-29 21:10:45,2019-02-14 03:06:31


In [6]:
path = f'{DATA_FOLDER}/graph_edges'
files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.find("embeddings_all") != -1])
files = files[0:2] + files[8:] + files[2:8]
files

['./data/mysql-2019-06-01//graph_edges/embeddings_all_0.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_1.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_2.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_3.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_4.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_5.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_6.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_7.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_8.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_9.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_10.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_11.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_12.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_13.v30.h5',
 './data/mysql-2019-06-01//graph_edges/embeddings_all_14.v30.h5',
 './data/mysql-2019-

next actions need some RAM!

In [7]:
with open(os.path.join(path, "dictionary.json"), "rt") as tf:
    dictionary = json.load(tf)
print(dictionary['relations'])
all_entities = dictionary['entities']['all']
len(all_entities)

['is_member_of', 'follows', 'relation', 'watches']


27406591

In [8]:
pd.Series(all_entities).map(lambda x: x.split('_id_')[0]).value_counts()

repo         16609312
user         10797277
entity_id           1
entity              1
dtype: int64

Read embeddings

In [9]:
embeddings=[]
for file in tn(files):
    with h5py.File(file, "r") as hf:
        embedding = hf["embeddings"][...]
    embeddings.append(embedding)
embeddings = np.concatenate(embeddings, axis=0)
embeddings.shape

(27406591, 100)

In [10]:
emb_df = pd.Series({i: e for i, e in tn(zip(all_entities, embeddings))}).to_frame(name='embeddings')
emb_df.shape

(27406591, 1)

Combine all metainfo with embeddings

In [11]:
repos_emb_df = pd.Series({int(i.split('_')[-1]): e for i, e in emb_df.loc[[f'repo_id_{i}' for i in repos_df.index]]['embeddings'].iteritems()}).to_frame(name='embeddings')
repos_df = repos_df.join(repos_emb_df)
repos_df

,url,language,created_at,updated_at,embeddings
_id,,,,,
6,cocos2d/cocos2d-x,C++,2010-11-18 23:17:00,2019-02-26 20:51:59,"[0.025527576, 0.2507348, -0.01867074, -0.28309..."
19,samuelclay/NewsBlur,Objective-C,2009-01-05 14:00:43,2019-02-26 03:37:41,"[-0.051254798, -0.10766631, -0.044019714, -0.0..."
29,opencv/opencv,C++,2012-07-19 09:40:17,2016-10-11 14:11:27,"[0.034408286, -0.07766469, -0.0800756, 0.02234..."
34,pockethub/PocketHub,JavaScript,2011-09-08 16:52:50,2016-10-11 14:11:27,"[0.2581099, -0.19753253, -0.0102489125, -0.151..."
37,angular/angular.js,JavaScript,2010-01-06 00:34:37,2019-02-27 03:14:26,"[0.052381046, -0.03020599, 0.01486524, -0.2154..."
49,alexgorbatchev/syntaxhighlighter,NaN,2014-10-07 13:48:58,2019-01-07 05:02:27,"[0.13156255, -0.07136197, 0.028045332, -0.1634..."
55,yui/yui3,JavaScript,2008-12-05 19:12:24,2019-02-26 04:45:11,"[0.16263631, 0.048907436, 0.13467114, -0.14233..."
71,llvm-mirror/llvm,NaN,2012-01-27 23:49:56,2019-02-26 12:59:44,"[0.15251222, 0.21407823, -0.1053832, 0.2561284..."
74,joyent/libuv,C,2011-03-29 21:10:45,2019-02-14 03:06:31,"[0.06940384, -0.03117697, 0.053432968, 0.16686..."


Save for TensorBoard

In [12]:
repos_df.to_pickle("./tb/embeddings/repos_19k_gte1k.pkl")